<h2> Chatbot Implementation </h2>
<br>
This is the classic case of a QnA machine based on answers, But this is all the basics that you need in order to build a chatbot! A context Based Chatbot!

How?

So in this case we pass in the question to get the answer! One thing that will improve the model which i did not do due to memory constraint was 'one-hot-encoding' of the output vector. Armed with One-Hot-Encoding the model is programmed to pick it up from there itself. 

Passing the Context!
Initially start with a 60 time step thing! with the first vector as having all values that have been asked lets say 

1) How are you? <-- this is the first statement hence in our 60 time steps vector, this becomes the step 1<br>
2) I am good what about you? <-- if this was the reply then in next step we will have 2 vectors with values in them and other 58 filled with 0s
<br>

Hence we can keep passing the context to the future time steps and so on and hence this becomes a context based chatbot, Also you can improve it with
<br>
1) Bidirectional LSTM<br>
2) Batch Normalization<br>
3) Dropout





In [1]:
import pandas as pd
from keras.preprocessing import sequence
import string
from nltk.tokenize import RegexpTokenizer
from sklearn.utils import shuffle
import copy
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras import Sequential
from keras.layers import RepeatVector
import numpy as np
from keras.layers import TimeDistributed
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
df = pd.read_table('WikiQA-train.tsv')
df = df[:10000]
df = shuffle(df)
df.head()

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label
1999,Q274,what happens at the end of toy story 3,D273,Toy Story 3,D273-14,It is currently the 10th-highest-grossing film...,0
4288,Q602,how many blind people are there in the us,D591,Blindness,D591-13,Blindness is defined as visual acuity of less ...,0
3598,Q499,how many rounds in boxing,D492,Boxing,D492-0,"Boxing (pugilism, prize fighting, the sweet sc...",0
3722,Q527,what happened in the 90's,D519,1990s,D519-1,It was the 10th and last decade of the 20th ce...,0
2212,Q298,what does a roman numeral L stand for?,D297,Roman numerals,D297-1,"Roman numerals, the numeric system in ancient ...",0


In [3]:
df['Question'] = df['Question'].str.lower()
df['Sentence'] = df['Sentence'].str.lower()

In [4]:
X = df['Question'].values
Y = df['Sentence'].values

In [5]:
word_dict = dict()
max_x = 0
max_y = 0
index = 1

In [6]:
tokenizer = RegexpTokenizer(r'\w+')
for i in range(len(X)):
    X[i] = tokenizer.tokenize(X[i])
    
    for j in range(len(X[i])):
        if X[i][j] in word_dict:
            X[i][j] = word_dict[X[i][j]]
        else:
            word_dict[X[i][j]] = index 
            X[i][j] = word_dict[X[i][j]]
            index = index + 1
        if len(X[i]) > max_x:
            max_x = len(X[i])
            temp_x = (len(X[i]),i)
        
    Y[i] = tokenizer.tokenize(Y[i])
    for k in range(len(Y[i])):
        if Y[i][k] in word_dict:
            Y[i][k] = word_dict[Y[i][k]]
        else:
            word_dict[Y[i][k]] = index 
            Y[i][k] = word_dict[Y[i][k]]
            index = index + 1
            
        if len(Y[i]) > max_y:
            max_y = len(Y[i])
            temp_y = (len(Y[i]), i)

In [7]:
words_inv_map = {v: k for k, v in word_dict.items()}

In [8]:
X_train = sequence.pad_sequences(X, maxlen=max_x,padding='post')
Y_train = sequence.pad_sequences(Y, maxlen=max_y,padding='post')

In [9]:
X_train

array([[   1,    2,    3, ...,    0,    0,    0],
       [  19,   20,   21, ...,    0,    0,    0],
       [  19,   20,   51, ...,    0,    0,    0],
       ...,
       [  19,   20, 2466, ...,    0,    0,    0],
       [  19,   20, 2155, ...,    0,    0,    0],
       [  19,  251,   11, ...,    0,    0,    0]], dtype=int32)

In [10]:
Y = []
for i in range(len(Y_train)):
    Y.append([[j] for j in Y_train[i]])
Y = np.asarray(Y)

In [11]:
(list(words_inv_map.keys())[-1])

14497

In [ ]:
#30913
model = Sequential()
model.add(Embedding(list(words_inv_map.keys())[-1], 32, input_length=X_train.shape[1], mask_zero=True ))
model.add(LSTM(100))

model.add(RepeatVector(Y_train.shape[1]))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(Y.shape[2])))                
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 19, 32)            463904    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 305, 100)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 305, 256)          365568    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 305, 1)            257       
Total params: 882,929
Trainable params: 882,929
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y, epochs=1000, verbose=2)

Epoch 1/1000
 - 69s - loss: -2.4352e+03 - acc: 0.0418
Epoch 2/1000
 - 68s - loss: -2.6863e+03 - acc: 1.7705e-05
Epoch 3/1000
 - 70s - loss: -2.6863e+03 - acc: 1.5738e-05
Epoch 4/1000


In [ ]:
np.around(model.predict(X_train))